In [1]:
using PyCall
@pyimport math
println(math.pi)

3.141592653589793


In [2]:
%load_ext fortranmagic

Unrecognized magic `%load_ext`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


In [3]:
using PyCall
@pyimport 
def py_calc_interim_cmd(cmd, param_d, rainfall):
    """Calculate interim CMD (M_{f}) in its linear form.

    :param cmd: float, current Catchment Moisture Deficit (M_{k})
    :param param_d: float, model parameter factor `d`
    :param rainfall: float, rainfall for current time step in mm

    :returns: float, interim CMD (M_{f})
    """
    if cmd < param_d:
        Mf = cmd * exp(-rainfall / param_d)
    elif cmd < (param_d + rainfall):
        Mf = param_d * exp((-rainfall + cmd - param_d) / param_d)
    else:
        Mf = cmd - rainfall
    # End if

    return Mf
# End calc_interim_cmd()

LoadError: [91mMethodError: no method matching @pyimport()[0m
Closest candidates are:
  @pyimport([91m::ANY[39m, [91m::Any...[39m) at C:\Users\u5121114\.julia\v0.6\PyCall\src\PyCall.jl:463[39m

In [4]:
function jl_calc_interim_cmd(cmd, param_d, rainfall)
    if cmd < param_d
        Mf = cmd * exp(-rainfall / param_d)
    elseif cmd < (param_d + rainfall)
        Mf = param_d * exp((-rainfall + cmd - param_d) / param_d)
    else
        Mf = cmd - rainfall
    end
        
    return Mf
end

jl_calc_interim_cmd (generic function with 1 method)

In [8]:
using BenchmarkTools, Compat

@btime jl_calc_interim_cmd(201.0, 194.0, 16.5)

  11.305 ns (0 allocations: 0 bytes)


184.72885233663575

In [6]:
function jl_opt_calc_interim_cmd(cmd::Float64, param_d::Float64, rainfall::Float64)::Float64
    local Mf::Float64
    if cmd < param_d
        Mf = cmd * exp(-rainfall / param_d)
    elseif cmd < (param_d + rainfall)
        Mf = param_d * exp((-rainfall + cmd - param_d) / param_d)
    else
        Mf = cmd - rainfall
    end
        
    return Mf
end

jl_opt_calc_interim_cmd (generic function with 1 method)

In [9]:
@btime jl_opt_calc_interim_cmd(201.0, 194.0, 16.5)

  11.669 ns (0 allocations: 0 bytes)


184.72885233663575